##Import Required Libraries.

In [ ]:
import ipywidgets as widgets
import io
from PIL import Image, ImageOps
from IPython.display import display,clear_output
from warnings import filterwarnings
import numpy as np
import pandas as pd
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from flask import Flask, render_template, request
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
import os
import sys
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
tf.keras.applications.InceptionV3
tf.compat.v1.keras.applications.InceptionV3
tf.compat.v1.keras.applications.inception_v3.InceptionV3

<function keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax')>

In [ ]:
tf.keras.applications.inception_v3.InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [ ]:
your_model_path = "/content/drive/MyDrive/heritage/Code/model.h5"

#to get the web link:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://imu6v15akd-496ff2e9c6d22116-5000-colab.googleusercontent.com/


# Run this cell and click the link on the cell above!
Please note that the cells below will continue to run until you stop them manually. Therefore, it is important that you allow these cell to run uninterrupted while you click the link provided above. If you interrupt the cell execution before clicking the link, the link will not display the expected output.

In [ ]:
batikProp = [["Batik Bali", "Bali, Indonesia", "Bali is much inspired by the beauty of nature, culture, stories of gods, and religious ceremonies in Bali."],
["Batik Dayak", "Kalimantan, Indonesia", "The term Dayak means river. So that this batik describes a variety of activities that are often related to the river."],
["Batik Geblek Renteng", "Special Region of Yogyakarta, Indonesia", "Renteng diambil dari susunan desain yang dibuat secara geometris semirip motif geometris parang."],
["Batik Ikat Celup", "Yogyakarta & Bali, Indonesia", "is the process of making motifs and colors on plain white cloth with the technique of tying and covering part of the fabric with rubber / raffia and granulated sugar plastic then dyed in color / fabric dye"],
["Batik Kawung", "Yogyakarta, Indonesia", "Kawung Batik is a batik motif whose shape is in the form of a circle similar to a kawung fruit which is neatly arranged geometrically."],
["Batik Lasem", "Jawa Tengah, Indonesia", "Batik Lasem is the result of the acculturation process of the Chinese community with Indonesia."],
["Batik Megamendung", "Jawa Barat, Indonesia", "The blue color symbolizes the color of the sky which is wide, friendly and calm and symbolizes the rain bearer who is awaited as a bearer of fertility and a giver of life."],
["Batik Parang", "Solo, Indonesia", "This parang batik has the meaning of advice to never give up, like ocean waves that never stop moving."],
["Batik Tambal", "Yogyakarta, Indoneisa", "The patchwork motif has the meaning of patching, meaning patching or repairing things that are damaged."]]
cultGeneralDesc = [['Batik','Batik is a process to give motifs to the media with various dyeing processes.']]

In [ ]:
from flask import Flask, render_template
# app = Flask(__name__, template_folder='drive/MyDrive/AI for Batik - Student Exchange/Batik Web/templates')
app = Flask(__name__,template_folder='/content/drive/MyDrive/heritage/Code/templates', static_folder="/content/drive/MyDrive/heritage/Code/static")

model = tf.keras.models.load_model("/content/drive/MyDrive/heritage/Code/model.h5")

@app.route("/")
def home():
    return render_template('index.html')

@app.route('/input',methods=['POST'])
def input():
    # imageFile for image preview on html
    imagefile = request.files['imageFile']
    imagefile.save("/content/drive/MyDrive/heritage/Code/static/image.jpg")

    image_path = "/content/drive/MyDrive/heritage/Code/static/image.jpg"

    # Function used for Transformation
    def load_and_transform(images, train = True):
        image = tf.io.read_file(images)
        image = tf.io.decode_jpeg(image , channels = 3)
        image = image / 255
        image = tf.image.resize(image, [224 , 224], method="nearest")
        image = tf.expand_dims(image , 0)
        return image

    image_test = load_and_transform(image_path)
    label_cat = model.predict(image_test)
    label = np.argmax(label_cat)

    # custom code
    # data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    # image = Image.open('/content/drive/MyDrive/AI for Batik - Student Exchange/Batik Web/static/image.jpg')
    # size = (224, 224)
    # image = ImageOps.fit(image, size, Image.ANTIALIAS)
    # image_array = np.asarray(image)
    # normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    # data[0] = normalized_image_array
    # prediction = model.predict(data)
    # label = np.argmax(prediction)

    # Variables for output
    className = batikProp[label][0]
    cultureName = className.split(' ')[0]
    origin = batikProp[label][1]
    desc = batikProp[label][2]

    # define function to find general description from culGeneralDesc array
    def cultGeneralDescFinder():
        for i in range(len(cultGeneralDesc)):
            if cultGeneralDesc[i][0] == cultureName:
                return cultGeneralDesc[i][1]

    return render_template('index.html',image = imagefile, cultureOutput=str(className), nameOutput=str(cultureName), cultureDesc= desc, cultureOrigin= origin, generalDescription=cultGeneralDescFinder())


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
